In [1]:
# !pip install dash

In [2]:
import pandas as pd
import plotly.express as px
import os
import json
import re
import numpy as np
import plotly.express as px
import json
from datetime import datetime

# Fetch the data

## Disaster Data

In [3]:
df = pd.read_excel('Data Bencana.xlsx')

In [4]:
df.head()

,No.,Kode Identitas Bencana,ID Kabupaten,Tanggal Kejadian,Kejadian,Lokasi,Kabupaten,Provinsi,Kronologi & Dokumentasi,Penyebab,Meninggal,Hilang,Terluka,Rumah Rusak,Rumah Terendam,Fasum Rusak
0,1,NaN,7504,2025-12-03,BANJIR,Kecamatan:\nKec. Marisa Ds. Bungalita Ds. Teratai,Pohuwato,Gorontalo,Dokumentasi,NaN,NaN,NaN,NaN,0.0,72.0,0
1,2,NaN,5205,2025-12-03,BANJIR,Kecamatan:\nKec. Woja Kel. Kandai Dua Kel. Mo...,Dompu,Nusa Tenggara Barat,Dokumentasi,NaN,NaN,NaN,NaN,0.0,251.0,0
2,3,NaN,3604,2025-12-03,CUACA EKSTREM,Kecamatan:\nKec. Kopo Ds. Nyompok,Serang,Banten,Dokumentasi,NaN,NaN,NaN,NaN,15.0,NaN,0
3,4,NaN,8205,2025-12-03,BANJIR,Kecamatan:\nKec. Sulabesi Ds. Bega,Kepulauan Sula,Maluku Utara,Dokumentasi,NaN,NaN,NaN,NaN,0.0,NaN,1
4,5,NaN,5206,2025-12-03,BANJIR,Kecamatan:\nKec. Bolo Ds. Rato Ds. Nggembe,Bima,Nusa Tenggara Barat,Dokumentasi,NaN,NaN,NaN,NaN,0.0,197.0,0


In [5]:
df.isna().sum()

,0
No.,0
Kode Identitas Bencana,1513
ID Kabupaten,0
Tanggal Kejadian,0
Kejadian,0
Lokasi,924
Kabupaten,0
Provinsi,0
Kronologi & Dokumentasi,0
Penyebab,11571


In [6]:
df['Kejadian'].value_counts()

,count
Kejadian,
BANJIR,9439
CUACA EKSTREM,7199
KEBAKARAN HUTAN DAN LAHAN,5280
TANAH LONGSOR,4591
KEKERINGAN,359
GEMPABUMI,303
GELOMBANG PASANG DAN ABRASI,246
ERUPSI GUNUNG API,33
TSUNAMI,2


## GeoJSON Data

In [7]:
# use github repo instead
link = 'https://github.com/JfrAziz/indonesia-district'
repo_name = link.split("/")[-1]
!rm -rf {repo_name}
!git clone {link}

Cloning into 'indonesia-district'...
remote: Enumerating objects: 8195, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 8195 (delta 4), reused 9 (delta 4), pack-reused 8184 (from 1)
Receiving objects: 100% (8195/8195), 411.07 MiB | 15.78 MiB/s, done.
Resolving deltas: 100% (136/136), done.
Updating files: 100% (7632/7632), done.


In [ ]:
repo_name = link.split("/")[-1]
!ls -R /content/{repo_name}

## Extract All of the Kecamatan data for later use

In [ ]:
repo_path = f'/content/{repo_name}'
set_kecamatan = []

for root, dirs, files in os.walk(repo_path):
  for file in files:
    if file.endswith('.geojson'):
      match = re.match(r'id\d+_(.+)\.geojson', file)
      if match:
        place_name = match.group(1)
        set_kecamatan.append(place_name)
set_kecamatan = list(set(set_kecamatan))


In [ ]:
print(f"Found {len(set_kecamatan)} unique places")
print("Sample places:", set_kecamatan[:10])

In [ ]:
set_kecamatan = [s.replace("_"," ") for s in set_kecamatan]
set_kecamatan

## Remove prefix id in the local directory

In [ ]:
def remove_id_prefix(name):
    return re.sub(r'^id', '', name)

In [ ]:
for root, dirs, files in os.walk(repo_path,topdown=False):
    for file in files:
        if file.startswith('id'):
            old_path = os.path.join(root, file)
            new_name = remove_id_prefix(file)
            new_path = os.path.join(root, new_name)
            os.rename(old_path, new_path)
            # print(f"File: {file} → {new_name}")

    for dir_name in dirs:
        if dir_name.startswith('id'):
            old_path = os.path.join(root, dir_name)
            new_name = remove_id_prefix(dir_name)
            new_path = os.path.join(root, new_name)
            os.rename(old_path, new_path)
            # print(f"Dir: {dir_name} → {new_name}")

In [ ]:
repo_name = link.split("/")[-1]
!ls -R /content/{repo_name}

# Data Preprocessing

## Filter Only The Needed Columns

In [ ]:
not_drop = ['Kabupaten','Provinsi', 'Lokasi','Tanggal Kejadian','Kejadian','ID Kabupaten']

In [ ]:
to_drop = [col for col in df.columns if col not in not_drop]

In [ ]:
to_drop

In [ ]:
df.drop(to_drop,axis=1,inplace=True)

In [ ]:
df.head()

## Add Missing ID's

### Extract all mapping of ids from the repo

In [ ]:
province_name_to_id = {}
province_id_to_name = {}
city_name_to_id = {}
city_id_to_name = {}
subdistrict_name_to_id = {}
subdistrict_id_to_name = {}

In [ ]:
for root, dirs, files in os.walk(repo_path):
    rel_path = os.path.relpath(root, repo_path)
    path_parts = rel_path.split(os.sep)

    if rel_path == '.':
        continue

    if len(path_parts) == 1:
        province = path_parts[0]
        if '_' in province:
            prov_id, prov_name = province.split('_', 1)
            province_name_to_id[prov_name] = prov_id
            province_id_to_name[prov_id] = prov_name

    elif len(path_parts) == 2:
        province = path_parts[0]
        city = path_parts[1]

        if '_' in city:
            city_id, city_name = city.split('_', 1)
            city_name_to_id[city_name] = city_id
            city_id_to_name[city_id] = city_name

    for file in files:
        if file.endswith('.geojson') and '_' in file:
            name = file.replace('.geojson', '')
            if '_' in name:
                subdistrict_id, subdistrict_name = name.split('_', 1)

                if len(subdistrict_id) > 4:
                    subdistrict_name_to_id[subdistrict_name] = subdistrict_id
                    subdistrict_id_to_name[subdistrict_id] = subdistrict_name

In [ ]:
print(f"Provinces: {len(province_name_to_id)}")
print(f"Cities: {len(city_name_to_id)}")
print(f"Subdistricts: {len(subdistrict_name_to_id)}")


### Add Missing Province ID

In [ ]:
df['Provinsi'].unique()

In [ ]:
df.loc[df['Provinsi'] == 'P A P U A', 'Provinsi'] = 'PAPUA'

In [ ]:
df.loc[:,'Provinsi'] = df['Provinsi'].str.lower()

In [ ]:
df.loc[:,'Provinsi'] = df['Provinsi'].str.replace(" ","_")


In [ ]:
province_name_to_id

In [ ]:
for prov in df['Provinsi'].unique():
  if not prov in province_name_to_id.keys():
    print(prov)

In [ ]:
# drop all papua for now
df = df[~df['Provinsi'].str.contains('papua', case=False, na=False)]

In [ ]:
df.loc[df['Provinsi'] == 'di_yogyakarta','Provinsi'] = 'daerah_istimewa_yogyakarta'

In [ ]:
df['Provinsi'].unique()

In [ ]:
df['ID Provinsi'] = df.loc[:,'Provinsi'].apply(lambda x : province_name_to_id[x])

### Create list of kecamatan column

In [ ]:
pd.set_option('display.max_rows', None)

# Show full column width
pd.set_option('display.max_colwidth', None)

print(df['Lokasi'].sample(n=10))

In [ ]:
kecamatan_names_normalized = {name.replace('_',' '): subdistrict_id for name, subdistrict_id in subdistrict_name_to_id.items()}
kecamatan_names_normalized

In [ ]:
def find_kecamatan(str_lokasi):
  if pd.isna(str_lokasi):
    return []

  str_lokasi_lower = str(str_lokasi).lower()

  matches = []
  for kec_name, kec_id in kecamatan_names_normalized.items():
    if kec_name.lower() in str_lokasi_lower:
      matches.append([kec_name,kec_id])

  return matches

In [ ]:
df['ID Kecamatan'] = df['Lokasi'].apply(find_kecamatan)

In [ ]:
df['ID Kecamatan'].head()

In [ ]:
df.head()

In [ ]:
df.drop(labels='Lokasi',inplace=True,axis=1)

In [ ]:
df.head()

In [ ]:
df = df.explode('ID Kecamatan').reset_index(drop=True)
df.head()

In [ ]:
df.head()

In [ ]:
def extract_kecamatan_name(x):
    try:
        if x is None or (isinstance(x, float) and np.isnan(x)):
            return None
        if isinstance(x, (list, np.ndarray)) and len(x) >= 2:
            return x[0]
        return None
    except:
        return None

def extract_kecamatan_id(x):
    try:
        if x is None or (isinstance(x, float) and np.isnan(x)):
            return None
        if isinstance(x, (list, np.ndarray)) and len(x) >= 2:
            return x[1]
        return None
    except:
        return None

In [ ]:
df['Kecamatan'] = df['ID Kecamatan'].apply(extract_kecamatan_name)
df['ID Kecamatan'] = df['ID Kecamatan'].apply(extract_kecamatan_id)


In [ ]:
df = df.drop_duplicates(keep='first').reset_index(drop=True)

In [ ]:
len(df)

In [ ]:
has_duplicates = df.duplicated().any()
has_duplicates

In [ ]:
df.head()

In [ ]:
df.sort_values(df.columns.tolist(), ignore_index=True).head()

## Download CSV

In [ ]:
df.to_csv('granular.csv')